In [1]:
import pandas as pd
df = pd.read_csv('data/processed/mimic_iii/diagnoses_noteevents_cleaned.csv')
df.head()

,TEXT,specialty,TEXT_final,TEXT_final_cleaned
0,Admission Date: [**2141-9-18**] ...,Cardiothoracic & Vascular,":\nhip pain\n\n:\n24yo woman with hx SLE, CKD(...",hip pain woman hx SLE CKD currently HD PD labi...
1,Admission Date: [**2187-9-19**] ...,Emergency Department,:\ns/p Motor cycle crash; left sided rib pain\...,Motor cycle crash left sided rib pain driver h...
2,Admission Date: [**2190-6-5**] Discharg...,Emergency Department,NaN,NaN
3,Admission Date: [**2101-4-30**] ...,Infectious Disease Specialty,:\nOSH transfer for sepsis\n\n:\n75 y/o M with...,sepsis hx type DM ESRD failed renal tx HD mont...
4,Admission Date: [**2146-9-15**] ...,Cardiothoracic & Vascular,":\nChest Pain, Abdominal Pain, Nausea/Vomiting...",Chest Pain Abdominal Pain Nausea Vomiting Righ...


In [5]:
df.specialty.nunique()

16

In [2]:
#show full text
pd.set_option('display.max_colwidth', -1)
df.head()

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1359/2299056610.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,version https://git-lfs.github.com/spec/v1
0,oid sha256:e58cfd03306f03486a5966363015ccc74c876450818e43a86192d46b978d0bf8
1,size 619203815


In [3]:
import os

import pandas as pd
import torch
import wandb
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers.trainer_utils import get_last_checkpoint

from src.nlp.constants import (
    MODEL_BASE_NAME,
    MODEL_TC_CHECKPOINTS_DIR,
    MODEL_TC_DIR,
    MTSAMPLES_PROCESSED_PATH_DIR,
)
from src.nlp.utils import (
    get_device,
    load_tokenizer,
    load_trainer,
    load_training_args,
    tokenize_function,
)
from src.nlp.text_classification_model_training import load_datasets, tokenize_dataset

# wandb.init(
#     project="nlp",
#     entity="nlp_masterthesis",
#     tags=["textclassification mimic"],
# )




2022-11-20 15:40:32.076165: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: taratumbraegel (nlp_masterthesis). Use `wandb login --relogin` to force relogin


In [20]:
def map_medical_specialty_to_labels(path: str) -> pd.DataFrame:
    """
    Read the csv file
    Map the medical specialty to labels

    Parameters
    ----------
    path : str
        Path to the csv file

    Returns
    -------
    pd.DataFrame
        Dataframe with the mapped labels
    """
    df = pd.read_csv(path)
    #drop the rows with missing values
    df = df.dropna()
    dict_medical_specialty = {
        value: idx for idx, value in enumerate(df.specialty.unique())
    }
    df["labels"] = df.specialty.map(dict_medical_specialty)
    return df




In [21]:
def load_datasets(data_path: str) -> tuple[Dataset, Dataset]:
    """
    Load the train and validation datasets

    Parameters
    ----------
    data_path : str
        Path to the dataset

    Returns
    -------
    tuple[Dataset, Dataset]
        train and validation datasets
    """
    dataset = Dataset.from_pandas(map_medical_specialty_to_labels(data_path))
    dataset_train_test = dataset.train_test_split(test_size=0.1)
    # train dataset
    dataset_train_val = dataset_train_test["train"].train_test_split(test_size=0.1)
    dataset_train = dataset_train_val["train"]
    # validation dataset
    dataset_val = dataset_train_val["test"]

    return dataset_train, dataset_val




In [22]:
train_ds, val_ds = load_datasets(
        os.path.join("data/processed/mimic_iii/diagnoses_noteevents_cleaned.csv")
    )

#use small sample for testing
train_ds = train_ds.select(range(10))
val_ds = val_ds.select(range(10))


In [23]:
def load_tokenizer() -> AutoTokenizer:
    """
    Load tokenizer

    Returns
    -------
    AutoTokenizer
        Tokenizer
    """
    tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=MODEL_BASE_NAME,
        model_max_length=512,
        truncate=True,
        max_length=512,
        padding=True,
    )

    return tokenizer

In [24]:
from datasets.arrow_dataset import Batch
def tokenize_function(
    batch: Batch, tokenizer: AutoTokenizer, special_token: bool
) -> Batch:
    """
    Tokenize the input batch

    Parameters
    ----------
    batch : Batch
        Batch to tokenize
    tokenizer : AutoTokenizer
        Tokenizer to use
    special_token : bool
        Whether to add special tokens

    Returns
    -------
    Batch
        Tokenized batch
    """
    # spcial_token = false for Text classification
    return tokenizer(
        batch["TEXT_final_cleaned"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_special_tokens_mask=False,
        #special_token,
    )

In [29]:
def tokenize_dataset(dataset: Dataset, tokenizer: AutoTokenizer) -> Dataset:
    """
    Tokenize the dataset

    Parameters
    ----------
    dataset : Dataset
        Dataset to tokenize
    tokenizer : AutoTokenizer
        Tokenizer

    Returns
    -------
    Dataset
        Tokenized dataset
    """
    # tokenize_dataset = dataset.map(
    #     lambda batch: tokenize_function(batch, tokenizer, False),
    #     batched=True,
    #     remove_columns=['TEXT', 'specialty', 'TEXT_final'],
    # )
    tokenized_datasets = dataset.map(
        tokenize_function,
        fn_kwargs={"tokenizer": tokenizer, "special_token": False},
        batched=True,
    )
    return tokenized_datasets

In [30]:

tokenizer = load_tokenizer()
#tokenized_train_ds = tokenize_dataset(train_ds, tokenizer)
#tokenized_val_ds = tokenize_dataset(val_ds, tokenizer)

In [14]:
train_ds

Dataset({
    features: ['TEXT', 'specialty', 'TEXT_final', 'TEXT_final_cleaned', 'labels'],
    num_rows: 10
})

In [31]:
tokenized_train_ds = tokenize_dataset(train_ds, tokenizer)
tokenized_val_ds = tokenize_dataset(val_ds, tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
def clean_remove_column(tokenized_dataset: Dataset) -> Dataset:
    """
    Remove all unneded columns from the dataset

    Parameters
    ----------
    tokenized_dataset : Dataset
        Tokenized dataset

    Returns
    -------
    Dataset
        Dataset with only the needed columns
    """
    print(tokenized_dataset.column_names)
    tokenized_dataset = tokenized_dataset.remove_columns(
        [
  'TEXT', 'specialty', 'TEXT_final'
        ]
    )
    # tokenized_dataset = tokenized_dataset.rename_column(
    #     "labels_val", "labels"
    # )
    tokenized_dataset.set_format("torch")
    return tokenized_dataset


def load_model(
    device: torch.device,
) -> AutoModelForSequenceClassification:
    """
    Load sequence classification model

    Parameters
    ----------
    device : torch.device
        Device

    Returns
    -------
    AutoModelForSequenceClassification
        Sequence classification model
    """

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_BASE_NAME, num_labels=16
    ).to(device)

    return model


def main() -> None:
    """
    Main function
    """

    train_ds, val_ds = load_datasets(
        os.path.join("../data/processed/mimic_iii/diagnoses_noteevents_cleaned.csv")
    )

    tokenizer = load_tokenizer()
    tokenized_train_ds = tokenize_dataset(train_ds, tokenizer)
    tokenized_val_ds = tokenize_dataset(val_ds, tokenizer)
    # only use sample size for testing
    tokenized_train_ds = tokenized_train_ds.select(range(10))
    tokenized_val_ds = tokenized_val_ds.select(range(10))

    tokenized_train_ds = clean_remove_column(tokenized_train_ds)
    tokenized_val_ds = clean_remove_column(tokenized_val_ds)

    # device = get_device()
    # model = load_model(device)
    # training_args = load_training_args(MODEL_TC_CHECKPOINTS_DIR)
    # trainer = load_trainer(
    #     model,
    #     training_args,
    #     tokenized_train_ds,
    #     tokenized_val_ds,
    #     tokenizer,
    #     modeltype="sequence_classification",
    # )

    # last_checkpoint = get_last_checkpoint(training_args.output_dir)
    # if last_checkpoint is None:
    #     resume_from_checkpoint = None
    # else:
    #     resume_from_checkpoint = True

    # trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    # trainer.save_model(MODEL_TC_DIR)
    # trainer.save_state()


if __name__ == "__main__":
    main()
